In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Import from the crewAI libray.
from crewai import Agent, Task, Crew


# import library
from dotenv import load_dotenv
import os #needed to call function
import openai

In [7]:
# setup configuration function
def configure():
    load_dotenv()
    
# load configuration
configure()

# get the key
openai_api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'


## crewAI Tools

In [8]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents
#### Crew that is able to organize a meetup


In [9]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [ ]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipment"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [11]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Pydantic will help to turn fuzzy outputs from LLM into strong typed outputs 
- Agents will populate this object with information about different venues by creating different instances of it.

In [12]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [13]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [15]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True, # stop and ask for human input
    async_execution=True, # run in parallel with other tasks
    agent=logistics_manager 
)

In [16]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [17]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [18]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [19]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to start by researching and finding a venue in San Francisco that meets the criteria for the Tech Innovation Conference.

Action: Search the internet
Action Input: {"search_query": "venues for conferences in San Francisco"} 


Search results: Title: San Francisco Meeting and Conference Venues - Presidio.gov
Link: https://presidio.gov/plan-an-event/meetings-and-conferences
Snippet: The Presidio offers some of San Francisco's most unique meeting and conference event spaces, with indoor and outdoor options.
---
Title: Conference Venues for Rent in San Francisco, CA - Tagvenue
Link: https://www.tagvenue.com/us/hire/conference-venues/san-francisco
Snippet: Tagvenue offers an extensive range of conference venues in San Francisco, making it easy for you to find the perfect one. Select and b



> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
I need to maximize event exposure and engage with potential attendees for the Tech Innovation Conference. 

Action: Read website content
Action Input: {"website_url": "www.techinnovationconference.com"} 

I encountered an error while trying to use the tool. This was the error: Invalid URL 'www.techinnovationconference.com': No scheme supplied. Perhaps you meant https://www.techinnovationconference.com?.
 Tool Read website content accepts these inputs: Read website content(website_url: 'string') - A tool that can be used to read a website content.

Thought: I need to maximize event exposure and engage with potential attendees for the Tech Innovation Conference. 

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference San Francisco"}I need to ensure that all logistics for the event, including catering and equipment setup, are flawlessly executed to create a seamless exp

- Display the generated `venue_details.json` file.

In [20]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'Park View Room, San Francisco, CA',
 'booking_status': 'Available',
 'capacity': 300,
 'name': 'San Francisco Italian Athletic Club'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [21]:
from IPython.display import Markdown
Markdown("marketing_report.md")

To effectively market the Tech Innovation Conference and engage potential attendees, it is crucial to leverage the insights and strategies shared at the San Francisco Innovation Conference. Attendees can gain valuable knowledge on future trends, innovative ideas, and practical tools to accelerate innovation in their businesses. The conference offers a unique opportunity to immerse in the future, learn actionable ideas, and thrive in an AI-enabled world. By attending, participants can access over 100 data-driven insights, downloadable presentations, individual assessments, and energetic sessions that focus on real-world applications. With top speakers like Jeremy Gutsche, Armida Ascano, and other industry leaders, the San Francisco Innovation Conference promises to be a transformative experience for all attendees. Attendees can expect to gain insights on various topics such as entertainment, leisure, post-pandemic scenarios, consumer behavior, retail trends, future tech like AI and automation, and much more. By attending this conference, participants can stay ahead of the curve, explore innovative hotspots in San Francisco, and gain a competitive edge in their respective industries.